# **Traffic Sign Recognition** 

---

**Build a Traffic Sign Recognition Project**

The goals / steps of this project are the following:
* Load the data set 
* Explore, summarize and visualize the data set
* Design, train and test a model architecture
* Use the model to make predictions on new images
* Analyze the softmax probabilities of the new images
* Summarize the results with a written report

## How to run
install conda, and create an environment based on the yml file provided
```bash
conda env create -f carnd-lenet.yml
```
activate the environment and run the jupyter notebook

```bash
activate carnd-clone
jupyter notebook Traffic_Sign_Classifier.ipynb
```
Download [German Traffic Sign Dataset](https://s3-us-west-1.amazonaws.com/udacity-selfdrivingcar/traffic-signs-data.zip), and extract to ../data/

the project is implemented in the Traffic_Sign_Classifier.ipynb

---


### Data Set Summary & Exploration

#### 1. Provide a basic summary of the data set. In the code, the analysis should be done using python, numpy and/or pandas methods rather than hardcoding results manually.

I used the numpy and built-in libraries to calculate summary statistics of the traffic
signs data set:

* The size of training set is 34799 samples.
* The size of the validation set is 4410 samples.
* The size of test set is 12630 samples.
* The shape of a traffic sign image is 32x32x3,
* The number of unique classes/labels in the data set is 43.

#### 2. Exploratory visualization of the dataset.

Here is an exploratory visualization of the data set. It is a bar chart showing how the data is distributed between classes. As it is shown in the figure below,training data, validation data, and test data are distributed similarly. However, the dataset distribution is imbalanced. this is the case for most of the real-life datasets. imbalanced dataset can raise the false negative predictions, resulting a lower f1 score for the model. one way to solve is over-sampling which is described later in the preprocessing stage. 

<figure>
<img id="DatasetDistribution" src="./res/dataset_distribution.png"/>
</figure>
The image below shows one random image from each class. as it can be seen, the images are taken in different lighting conditions, some are obscured, and some are taken from the different angels. this would result in the generlized model which would classify the new images more accurately.
<figure>
<img id="RandomImages" src="./res/random_images.png"/>
</figure>


### Design and Test a Model Architecture

#### 1. Pre-processing

As it can be seen in <a href="./res/dataset_distribution.png">dataset distribution</a>, the some classes certainly lack samples. for instance, for "Speed limit 20 km/h" class (class 0), the model has to classify 60 images correctly after seeing only 180 images. 

one way to add more data is to barrow samples from other classes, convert them so that they look like distenation class samples. this can be done by "fipped" vertically or horizontally. for example, one sample from "go straight or left" class can be easily mirrored horizontally and turned into "go straght or right" sample and vice versa. this, also, works for the following pairs: "Dangerous curve to the left" and "Dangerous curve to the right", "turn right ahead" and "turn left ahead", "keep right" and "keep left". 

<figure>
<img src="./res/flipped_img.png"/>
</figure>

The next step is transform the images in same class to get new samples. this is done by adding random noise to image, then rotating the image by a random degree between 0 and 10 around a random point at the center, and finally making a random transtion on the image. also, samples from some traffic sign images can be flipped vertically and/or horizontally and still keep their meaning, such as "No Entry" sign flipped vertically and horizontally has the same meaning. this is, also, applied to the images of those classes by random to create new samples.

<figure>
<img src="./res/transformed_img.png"/>
</figure>

at the end, the number of images of each class reached to 2500 samples, making 107500 samples, of which 4079 samples are taken from other classes, and 68622 samples are synthetic.

<figure>
<img src="./res/augmented_data.png"/>
</figure>


Next, I decided to convert the images to grayscale because first this conversion reduces the amount of input data to the model and expedites the training. second, the traffic sign images have different colors at the boarders and also in different lighting conditions colors might look different, the grayscale image helps to eliminate the dependency on the color. third, based on [this paper](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf), the accuracy of the model increases by ignoring the color. 

Here is an example of a traffic sign image before and after grayscaling.

<figure>
<img src="./res/gray_scale.png"/>
</figure>

As a last step, I normalized the image data because it prevent the weights and biases to get too big or too small; therefore, it will help the optimizer to find the numerical solution faster.



#### 2. Model architecture 

 The final model looks like the sermanet except the size of the laters are different:

<figure>
<img src="./res/sermanet.png"/>
</figure>

My final model consisted of the following layers:

| Layer         		|     Description	        					| 
|:---------------------:|:---------------------------------------------:| 
| Input         		| 32x32x1 Grayscale image   					| 
| Convolution 5x5     	| 1x1 stride, valid padding, outputs 28x28x54 	|
| RELU					|												|
| Max pooling 2x2      	| 2x2 stride,  outputs 14x14x54 				|
| Dropout           	|                               				|
| Convolution 5x5	    | 1x1 stride, valid padding, outputs 10x10x100	|
| RELU					|												|
| Max pooling 2x2       | 2x2 stride,  outputs 5x5x100 				    |
| Dropout           	|                               				|
| Fully connected		| 120 neurons  									|
| Dropout           	|                               				|
| Fully connected		| 84 neurons  									|
| Dropout           	|                               				|
| Fully connected		| 43 neurons  									|
| Softmax				|           									|
|       				|           									|
 


#### 3. Training the model. 

To train the model, I used Adam optimizer and 128 for the batch size as it was used in the LeNet. I increased number of epochs to 100 and I chose the learning to be 0.0005. 

#### 4. Discusstion on the model performance

My final model results were:
* training set accuracy of ?
* validation set accuracy of ? 
* test set accuracy of ?

I started from the LeNet architecture since its performance is fairly good on the MNIST dataset and traffic sign dataset, although more complicated, is similar to the MNIST. 

I chose to add more complexity to the LeNet model to make it suitable for the traffic sign classifier. By increasing the convolution layers filter count, the model can recognize more details at the first stage, and more shapes at the second stage. with this modification, the model accuracy on test dataset reached to 94 percent. However, it didn't seem satisfying to me because the permforamance on the new test images was dismal (around 40 percent). 

Therefore, i decided to add dropout regulization to prevent the overfitting since the accuracy on the training and validation set were above 98 percent. this suprisingly decreased the accuracy. i realized since half of the parameter are set to zero randomly, the training must continue for more epochs. by increasing the number of epochs and lowering the learning rate, the accuracy on the test dataset increased to 96 percent. also, based on the [sermanet](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf) i added a subsampling from the first stage to the fully connected classifier.
 

### Test a Model on New Images

#### 1. Discusstion on images

Here are ten traffic signs that I found on the [STSD](https://www.cvl.isy.liu.se/research/datasets/traffic-signs-dataset/):

<figure>
<img src="./res/extra_images.png"/>
</figure>

 the Swedish traffic signs are quite similar to the German ones; however, most of the signs have yellow-background. also, the stop sign is from a different angle. keep left sign is covered with some horizontal lines. Other images are taken in different lighting condition.


#### 2. Discuss the model's predictions on these new traffic signs and compare the results to predicting on the test set. At a minimum, discuss what the predictions were, the accuracy on these new predictions, and compare the accuracy to the accuracy on the test set (OPTIONAL: Discuss the results in more detail as described in the "Stand Out Suggestions" part of the rubric).

Here are the results of the prediction:

| Image			        |     Prediction	        					| 
|:---------------------:|:---------------------------------------------:| 
| Stop Sign      		| Stop sign   									| 
| U-turn     			| U-turn 										|
| Yield					| Yield											|
| 100 km/h	      		| Bumpy Road					 				|
| Slippery Road			| Slippery Road      							|


The model was able to correctly guess 4 of the 5 traffic signs, which gives an accuracy of 80%. This compares favorably to the accuracy on the test set of ...

#### 3. Describe how certain the model is when predicting on each of the five new images by looking at the softmax probabilities for each prediction. Provide the top 5 softmax probabilities for each image along with the sign type of each probability. (OPTIONAL: as described in the "Stand Out Suggestions" part of the rubric, visualizations can also be provided such as bar charts)

The code for making predictions on my final model is located in the 11th cell of the Ipython notebook.

For the first image, the model is relatively sure that this is a stop sign (probability of 0.6), and the image does contain a stop sign. The top five soft max probabilities were

| Probability         	|     Prediction	        					| 
|:---------------------:|:---------------------------------------------:| 
| .60         			| Stop sign   									| 
| .20     				| U-turn 										|
| .05					| Yield											|
| .04	      			| Bumpy Road					 				|
| .01				    | Slippery Road      							|


For the second image ... 

### (Optional) Visualizing the Neural Network (See Step 4 of the Ipython notebook for more details)
#### 1. Discuss the visual output of your trained network's feature maps. What characteristics did the neural network use to make classifications?
